## ДЗ 8. Рекуррентные нейронные сети RNN LSTM GRU

### Задание
Данные берем отызывы за лето.

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи.

1. построить свёрточные архитектуры 
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [154]:
!pip install pymorphy2
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [155]:
# Импорт библиотек

import pandas as pd
import numpy as np
import re
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from gensim.models import Word2Vec, FastText

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, GlobalMaxPool2D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

from sklearn.metrics import classification_report

In [156]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [157]:
df = pd.read_excel('отзывы за лето.xls')

In [158]:
df

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [159]:
df = df.dropna()

In [160]:
df.shape

(20656, 3)

In [161]:
df.Rating.value_counts()

5    14584
1     2276
4     2137
3      911
2      748
Name: Rating, dtype: int64

In [162]:
# делаем колонку Content текстовой - так как ругается что - 'int' object has no attribute 'translate'

df.Content = df.Content.apply(lambda text: str(text))

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [163]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Content, df.Rating, test_size=0.3, random_state=42, stratify=df.Rating)

In [164]:
y_train.value_counts()

5    10209
1     1593
4     1496
3      638
2      523
Name: Rating, dtype: int64

In [165]:
y_test.value_counts()

5    4375
1     683
4     641
3     273
2     225
Name: Rating, dtype: int64

In [166]:
corpus_train = X_train
corpus_test = X_test

In [167]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(corpus_train)

sequences_train = tokenizer.texts_to_sequences(corpus_train)
sequences_test = tokenizer.texts_to_sequences(corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

## Построение сетей

In [169]:
num_classes = 6

epochs = 20
batch_size = 512

comparing_list = []

In [170]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

### CNN+pooling

In [171]:
net_name = 'CNN+pooling'

model = Sequential()
model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
# model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping],
                    )

Epoch 1/20
26/26 [==============================] - 26s 984ms/step - loss: 1.4921 - accuracy: 0.6258 - val_loss: 1.0924 - val_accuracy: 0.7026
Epoch 2/20
26/26 [==============================] - 18s 707ms/step - loss: 0.9608 - accuracy: 0.7065 - val_loss: 0.8505 - val_accuracy: 0.7047


In [172]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.11      1.00      0.20       683
           2       0.00      0.00      0.00       225
           3       0.00      0.00      0.00       273
           4       0.10      1.00      0.19       641
           5       0.71      1.00      0.83      4375

   micro avg       0.31      0.92      0.46      6197
   macro avg       0.15      0.50      0.20      6197
weighted avg       0.52      0.92      0.63      6197
 samples avg       0.31      0.92      0.46      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RNN

In [173]:
net_name = 'RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 14s 480ms/step - loss: 1.4800 - accuracy: 0.6082 - val_loss: 1.0341 - val_accuracy: 0.7241
Epoch 2/20
26/26 [==============================] - 12s 467ms/step - loss: 0.8569 - accuracy: 0.7469 - val_loss: 0.7657 - val_accuracy: 0.7656


In [174]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.40      0.80      0.53       683
           2       0.17      0.38      0.24       225
           3       0.18      0.25      0.21       273
           4       0.10      0.83      0.19       641
           5       0.80      0.98      0.88      4375

   micro avg       0.43      0.89      0.58      6197
   macro avg       0.28      0.54      0.34      6197
weighted avg       0.63      0.89      0.72      6197
 samples avg       0.48      0.89      0.61      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CNN + RNN

In [175]:
net_name = 'CNN+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3, activation='relu'))
# model.add(GlobalMaxPool1D(keepdims=True))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

# print(model.summary())
# plot_model(model, show_shapes=True, show_layer_names=True)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 32s 1s/step - loss: 1.1385 - accuracy: 0.6773 - val_loss: 0.8666 - val_accuracy: 0.7178
Epoch 2/20
26/26 [==============================] - 25s 975ms/step - loss: 0.8557 - accuracy: 0.7394 - val_loss: 0.7506 - val_accuracy: 0.7545


In [176]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.21      0.96      0.35       683
           2       0.15      0.48      0.23       225
           3       0.31      0.01      0.03       273
           4       0.10      0.97      0.19       641
           5       0.77      0.98      0.86      4375

   micro avg       0.37      0.92      0.52      6197
   macro avg       0.26      0.57      0.27      6197
weighted avg       0.59      0.92      0.68      6197
 samples avg       0.39      0.92      0.54      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CNN+pooling+RNN

In [177]:
net_name = 'CNN+pooling+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D(keepdims=True))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

# print(model.summary())
# plot_model(model, show_shapes=True, show_layer_names=True)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 20s 726ms/step - loss: 1.3195 - accuracy: 0.6486 - val_loss: 0.9213 - val_accuracy: 0.7026
Epoch 2/20
26/26 [==============================] - 19s 726ms/step - loss: 0.8860 - accuracy: 0.7240 - val_loss: 0.7103 - val_accuracy: 0.7573


In [178]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.35      0.85      0.50       683
           2       0.00      0.00      0.00       225
           3       0.15      0.46      0.22       273
           4       0.10      0.94      0.19       641
           5       0.76      0.99      0.86      4375

   micro avg       0.40      0.91      0.56      6197
   macro avg       0.23      0.54      0.29      6197
weighted avg       0.59      0.91      0.69      6197
 samples avg       0.42      0.91      0.57      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RNN + CNN + CNN

In [179]:
net_name = 'RNN+CNN+CNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
model.add(SimpleRNN(64, recurrent_dropout=0.2, return_sequences="True"))
model.add(Conv1D(64, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(num_classes, activation="sigmoid"))      


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 24s 867ms/step - loss: 0.3821 - accuracy: 0.6757 - val_loss: 0.2327 - val_accuracy: 0.7227
Epoch 2/20
26/26 [==============================] - 22s 854ms/step - loss: 0.2237 - accuracy: 0.7360 - val_loss: 0.2075 - val_accuracy: 0.7490


In [180]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.48      0.14      0.22       683
           2       1.00      0.00      0.01       225
           3       0.00      0.00      0.00       273
           4       0.00      0.00      0.00       641
           5       0.84      0.94      0.89      4375

   micro avg       0.83      0.68      0.75      6197
   macro avg       0.39      0.18      0.19      6197
weighted avg       0.69      0.68      0.65      6197
 samples avg       0.68      0.68      0.68      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RNN CNN pooling RNN

In [181]:
net_name = 'RNN+CNN+pooling+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D(keepdims=True)) # keepdims Сохраняет размерность для SimpleRNN
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 41s 2s/step - loss: 1.2378 - accuracy: 0.6963 - val_loss: 0.7673 - val_accuracy: 0.7400
Epoch 2/20
26/26 [==============================] - 39s 1s/step - loss: 0.7588 - accuracy: 0.7481 - val_loss: 0.7701 - val_accuracy: 0.7580


In [182]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.33      0.91      0.48       683
           2       0.14      0.77      0.23       225
           3       0.15      0.59      0.24       273
           4       0.10      1.00      0.19       641
           5       0.71      1.00      0.83      4375

   micro avg       0.36      0.96      0.52      6197
   macro avg       0.24      0.71      0.33      6197
weighted avg       0.56      0.96      0.68      6197
 samples avg       0.41      0.96      0.56      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## (CNN RNN)x3 + polling

In [183]:
net_name = '(CNN+RNN)x3+polling'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3, activation='relu'))
model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(64, 3, activation='relu'))
model.add(SimpleRNN(64, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(32, 2, activation='relu'))
model.add(SimpleRNN(32, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 54s 2s/step - loss: 1.2669 - accuracy: 0.6237 - val_loss: 0.9949 - val_accuracy: 0.7026
Epoch 2/20
26/26 [==============================] - 52s 2s/step - loss: 1.0389 - accuracy: 0.7003 - val_loss: 0.8695 - val_accuracy: 0.7026


In [184]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.11      1.00      0.20       683
           2       0.00      0.00      0.00       225
           3       0.04      1.00      0.08       273
           4       0.10      1.00      0.19       641
           5       0.71      1.00      0.83      4375

   micro avg       0.24      0.96      0.39      6197
   macro avg       0.16      0.67      0.22      6197
weighted avg       0.52      0.96      0.63      6197
 samples avg       0.24      0.96      0.39      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Сравнение результатов

In [185]:
comparing_list

[['CNN+pooling', 0.7065242528915405, 0.704702615737915],
 ['RNN', 0.7468684911727905, 0.7655601501464844],
 ['CNN+RNN', 0.7394144535064697, 0.7544951438903809],
 ['CNN+pooling+RNN', 0.7240451574325562, 0.7572613954544067],
 ['RNN+CNN+CNN', 0.7359563708305359, 0.7489626407623291],
 ['RNN+CNN+pooling+RNN', 0.7480980753898621, 0.7579529881477356],
 ['(CNN+RNN)x3+polling', 0.7002996802330017, 0.702627956867218]]

In [186]:
pd.DataFrame(comparing_list, columns=['net_name', 'accuracy', 'val_accuracy'])

,net_name,accuracy,val_accuracy
0,CNN+pooling,0.706524,0.704703
1,RNN,0.746868,0.765560
2,CNN+RNN,0.739414,0.754495
3,CNN+pooling+RNN,0.724045,0.757261
4,RNN+CNN+CNN,0.735956,0.748963
5,RNN+CNN+pooling+RNN,0.748098,0.757953
6,(CNN+RNN)x3+polling,0.700300,0.702628


как ни смешно, но обычная RNN уделала остальные архитектуры по метрике качества. Она же относительно других архитектур неплохо предсказывает и классы кроме 5 это (1,2,3,4)